In [62]:
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
from dbscrape import *
# Import the backtrader platform
import backtrader as bt

data1 = gettablerange("db.pikujs.com","ohlcvdata","postgres","timepa$$","banknifty_f1","2020-01-01 09:16:00","2020-07-31 12:16:00")
pd.to_datetime(data1['datetime'])
data1 = data1.set_index('datetime')

data1 = data1[['open','high','low','close','volume']]


class Streak(bt.ind.PeriodN):
    '''
    Keeps a counter of the current upwards/downwards/neutral streak
    '''
    lines = ('streak',)
    params = dict(period=5) # need prev/cur days (2) for comparisons

    curstreak = 0

    def next(self):
        d0, d1 = self.data[0], self.data[-1]

        if d0 > d1:
            self.l.streak[0] = self.curstreak = max(1, self.curstreak + 1)
        elif d0 < d1:
            self.l.streak[0] = self.curstreak = min(-1, self.curstreak - 1)
        else:
            self.l.streak[0] = self.curstreak = 0


class ConnorsRSI(bt.Indicator):

    lines = ('crsi',)
    params = dict(prsi=3, pstreak=2, prank=100)

    def __init__(self):
        # Calculate the components
        rsi = bt.ind.RSI(self.data, period=self.p.prsi)
        streak = Streak(self.data)
        rsi_streak = bt.ind.RSI(streak.data, period=self.p.pstreak)
        prank = bt.ind.PercentRank(self.data, period=self.p.prank)

        # Apply the formula
        self.l.crsi = (rsi + rsi_streak + prank) / 3.0



# Create a Stratey
class TestStrategy(bt.Strategy):
    
    params = (('BBandsperiod', 20),)

    def log(self, txt, dt=None,dt1 =None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        dt1 = dt1 or self.datas[0].datetime.time(0)
        print('%s %s, %s' % (dt.isoformat(), dt1.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.redline = None
        self.blueline = None
        self.myind = ConnorsRSI()

        # Add a BBand indicator
        self.bband = bt.indicators.BBands(self.datas[0], period=self.params.BBandsperiod,devfactor = 2.0)
        self.bband1 = bt.indicators.BBands(self.datas[0], period=self.params.BBandsperiod,devfactor = 3.0)
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enougth cash
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    

    def next(self):
        # Simply log the closing price of the series from the reference
        #self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        if self.dataclose < self.bband.lines.bot and not self.position:
                self.redline = True

        if self.dataclose > self.bband.lines.top and self.position:
                self.blueline = True

        if self.dataclose > self.bband.lines.mid and not self.position and self.redline and self.myind.crsi[0] <= 30:        	
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        if self.dataclose > self.bband.lines.top and not self.position and self.myind.crsi[0] <= 30:
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
   
        if self.position and ( (self.dataclose > self.bband1.lines.top) or ((self.dataclose- self.buyprice)>-50) ):
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.blueline = False
                self.redline = False
                self.order = self.sell()


In [63]:

if __name__ == '__main__':
# Create a cerebro entity
    cerebro = bt.Cerebro()
    
    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    # Create a Data Feed
    data = bt.feeds.PandasData(dataname=data1, fromdate=datetime.datetime(2020,5,1), todate=bt.datetime.datetime(2020,7,31), timeframe=bt.TimeFrame.Days, compression=1)
    
    # Add the Data Feed to Cerebro
    cerebro.adddata(data)
    
    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    
    cerebro.addsizer(bt.sizers.FixedSize, stake=1)
    # Set the commission
    #cerebro.broker.setcommission(commission=0.001)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    # Run over everything
    cerebro.run()
    
    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


Starting Portfolio Value: 100000.00
2020-05-04 11:43:00, BUY CREATE, 19879.45
2020-05-04 11:44:00, BUY EXECUTED, Price: 19879.05, Cost: 19879.05, Comm 0.00
2020-05-04 11:44:00, SELL CREATE, 19855.30
2020-05-04 11:45:00, SELL EXECUTED, Price: 19856.80, Cost: 19879.05, Comm 0.00
2020-05-04 11:45:00, OPERATION PROFIT, GROSS -22.25, NET -22.25
2020-05-04 12:42:00, BUY CREATE, 19875.00
2020-05-04 12:43:00, BUY EXECUTED, Price: 19880.00, Cost: 19880.00, Comm 0.00
2020-05-04 12:43:00, SELL CREATE, 19880.00
2020-05-04 12:44:00, SELL EXECUTED, Price: 19885.00, Cost: 19880.00, Comm 0.00
2020-05-04 12:44:00, OPERATION PROFIT, GROSS 5.00, NET 5.00
2020-05-04 13:15:00, BUY CREATE, 19825.80
2020-05-04 13:16:00, BUY EXECUTED, Price: 19820.00, Cost: 19820.00, Comm 0.00
2020-05-04 13:16:00, SELL CREATE, 19829.05
2020-05-04 13:17:00, SELL EXECUTED, Price: 19830.20, Cost: 19820.00, Comm 0.00
2020-05-04 13:17:00, OPERATION PROFIT, GROSS 10.20, NET 10.20
2020-05-04 14:58:00, BUY CREATE, 19588.40
2020-05-04